In [1]:
import os
from pathlib import Path
import sys

import torch

rootDir = Path(os.path.abspath(__name__)).parent.parent
sys.path.append(str(rootDir))

from commsimcnn.util import *
from commsimcnn.models import TinyVGG

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

print(torch.__version__)
print(torchvision.__version__)

2.9.1+cu126
0.24.1+cu126


In [4]:
# Setup training data
train_data = datasets.FashionMNIST(
    root="data", # where to download data to
    train=True, # Do we want the training dataset
    download=True, # Do we want to download
    transform=ToTensor(), # How do we want to transform the data
    target_transform=None # how do we want to transform the labels/targets
)

# Setup testing data
test_data = datasets.FashionMNIST(
    root="data", # where to download data to
    train=False, # Do we want the training dataset
    download=True, # Do we want to download
    transform=ToTensor(), # How do we want to transform the data
)

In [5]:
from torch.utils.data import DataLoader

# Setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

# For test data, generally good to not shuffle (easier for evaluation)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x71cff37a2570>,
 <torch.utils.data.dataloader.DataLoader at 0x71cffee12300>)

In [6]:
mod0 = TinyVGG(input_shape=torch.Size([32, 1, 28, 28]),
               output_shape=10,
               in_channels=1).to(device)         

Output Height: 4 | Output Width: 4


In [7]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device = device):
  """ Performs a training step with a model trying to learn on data_loader"""
  train_loss, train_acc = 0, 0
  model.to(device)

  # Put model into training mode
  model.train()

  # Add a loop to loop through the training batches
  for batch, (X, y) in enumerate(data_loader):

    #Put data on target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Calculate loss (per batch)
    loss = loss_fn(y_pred, y)
    train_loss += loss # accumulate train loss

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

  # Divide total train loss and acc by length of train dataloader
  train_loss /= len(data_loader)
  print(f"Train loss: {train_loss:.5f}\n")

In [8]:
def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device = device):
  """Performs a testing loop step on model going over data_loader."""

  test_loss, test_acc = 0, 0
  model.to(device)

  # Put the model in eval mode
  model.eval()

  # Turn on inference mode context manager
  with torch.inference_mode():
    for X, y in data_loader:

      # Send data to the target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred = model(X)

      # 2. Calculate loss (accumulatively)
      test_loss += loss_fn(test_pred, y)

    # Calculate the test loss average per batch
    test_loss /= len(data_loader)

  # Print out what's happening
  print(f"Test loss: {test_loss:.5f}\n")

In [9]:
# Loss function/eval metrics/optimizer
from torch import nn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mod0.parameters(),
                            lr=0.1)

In [10]:
from timeit import default_timer as timer

def print_train_time(start: float,
                     end: float,
                     device: torch.device = None):
  """Prints difference between start and end time."""

  total_time = end - start
  print(f"Train time on {device}: {total_time:.3f} seconds")
  return total_time

In [11]:
from tqdm.auto import tqdm

torch.manual_seed(42)

# Measure time
from timeit import default_timer as timer
train_time_start = timer()

# Train and test model
epochs = 3
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n----------")
  train_step(model=mod0,
             data_loader=train_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             device=device)
  test_step(model=mod0,
             data_loader=test_dataloader,
             loss_fn=loss_fn,
             device=device)

train_time_end = timer()
total_train_time = print_train_time(start=train_time_start,
                                            end=train_time_end)

/home/trevor/dev/CommSimCNN/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
----------
Train loss: 0.73749



 33%|███▎      | 1/3 [00:02<00:05,  2.56s/it]

Test loss: 0.47832

Epoch: 1
----------
Train loss: 0.42598



 67%|██████▋   | 2/3 [00:04<00:02,  2.44s/it]

Test loss: 0.41000

Epoch: 2
----------
Train loss: 0.38092



100%|██████████| 3/3 [00:07<00:00,  2.43s/it]

Test loss: 0.37753

Train time on None: 7.302 seconds
